<a href="https://colab.research.google.com/github/AlbertoVentura8/LimpiezaDatos/blob/main/MarchMad_Limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**En este apartado, me voy a centrar en la limpieza del dataset.**


1.   Voy a dar acceso a un drive donde está almacenado y donde almacenaremos el
resultado del dataset.
2.   Eliminaremos una serie de columnas que hacen que el conjunto de datos tenga una dimensión poco manejable, y lo vamos a reducir para que sea tratable y poder sacar algunas conclusiones.

In [1]:
# Vamos a montar el drive, lo primero es
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Librerías que utilizaremos
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

In [3]:
#Hacemos la carga del dataset. Dataset previamente subido a GoogleColab.
NCAA_df = pd.read_csv('https://drive.google.com/uc?export=download&id=1pkvH-wKa3nLWcquQNNv4GxcDfMc3NOwo')
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 144)


,Season,Team Name,Conference,Historical Conference,Adjusted Temo,Adjusted Tempo Rank,Raw Tempo,Raw Tempo Rank,Adjusted Offensive Efficiency,Adjusted Offensive Efficiency Rank,...,RankPGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating,Net Rating Rank,Seed,Region,Post-Season Tournament,Post-Season Tournament Sorting Index
0,2015,Kentucky,Southeastern Conference (SEC),Southeastern Conference (SEC),62.4,274,63.8,242,121.3,6,...,311.0,John Calipari,14 years,15.0,36.9,1,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
1,2021,Gonzaga,West Coast Conference (WCC),West Coast Conference (WCC),73.8,7,74.3,14,126.4,1,...,20.0,Mark Few,24 years,25.0,36.5,2,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
2,2024,UConn,Big East Conference,Big East Conference,64.6,330,66.0,305,127.5,1,...,126.0,Dan Hurley,5 years,6.0,36.4,3,1,East,March Madness,1
3,2008,Kansas,Big 12 Conference,Big 12 Conference,66.8,109,68.5,120,120.9,2,...,209.0,Bill Self,20 years,21.0,35.2,4,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
4,2019,Virginia,Atlantic Coast Conference (ACC),Atlantic Coast Conference (ACC),59.4,353,60.6,353,123.4,2,...,196.0,Tony Bennett,14 years,15.0,34.2,5,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5


In [4]:
#Primero vamos a ver que columnas tienen mas celdas vacias
df_null_counts = NCAA_df.isnull().sum()
print(df_null_counts)


Season                                    0
Team Name                               253
Conference                              253
Historical Conference                   176
Adjusted Temo                             0
                                       ... 
Net Rating Rank                           0
Seed                                      0
Region                                    0
Post-Season Tournament                    0
Post-Season Tournament Sorting Index      0
Length: 144, dtype: int64


In [5]:
#Vamos a eliminar las filas que no tienen nombre de equipo
NCAA_df = NCAA_df.dropna(subset=['Team Name'])
print("Tamaño del dataset:", NCAA_df.shape)
df_null_counts = NCAA_df.isnull().sum()
print(df_null_counts)


Tamaño del dataset: (7697, 144)
Season                                    0
Team Name                                 0
Conference                                0
Historical Conference                   164
Adjusted Temo                             0
                                       ... 
Net Rating Rank                           0
Seed                                      0
Region                                    0
Post-Season Tournament                    0
Post-Season Tournament Sorting Index      0
Length: 144, dtype: int64


De momento los vacios de Historical Conference no nos afectan porque más adelante eliminaremos la columna.

In [6]:
#En esta línea eliminamos todas la columnas que tengan la palabra Raw, para reducir el dataset inicial.
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Raw')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()


Tamaño del dataset: (7697, 138)


,Season,Team Name,Conference,Historical Conference,Adjusted Temo,Adjusted Tempo Rank,Adjusted Offensive Efficiency,Adjusted Offensive Efficiency Rank,Adjusted Defensive Efficiency,Adjusted Defensive Efficiency Rank,...,RankPGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating,Net Rating Rank,Seed,Region,Post-Season Tournament,Post-Season Tournament Sorting Index
0,2015,Kentucky,Southeastern Conference (SEC),Southeastern Conference (SEC),62.4,274,121.3,6,84.4,1,...,311.0,John Calipari,14 years,15.0,36.9,1,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
1,2021,Gonzaga,West Coast Conference (WCC),West Coast Conference (WCC),73.8,7,126.4,1,89.9,11,...,20.0,Mark Few,24 years,25.0,36.5,2,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
2,2024,UConn,Big East Conference,Big East Conference,64.6,330,127.5,1,91.1,4,...,126.0,Dan Hurley,5 years,6.0,36.4,3,1,East,March Madness,1
3,2008,Kansas,Big 12 Conference,Big 12 Conference,66.8,109,120.9,2,85.7,1,...,209.0,Bill Self,20 years,21.0,35.2,4,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
4,2019,Virginia,Atlantic Coast Conference (ACC),Atlantic Coast Conference (ACC),59.4,353,123.4,2,89.2,5,...,196.0,Tony Bennett,14 years,15.0,34.2,5,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5


In [7]:
#Hacemos lo mismo que en la línea anterior para eliminar los rankings.
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Rank')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7697, 75)


,Season,Team Name,Conference,Historical Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,...,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating,Seed,Region,Post-Season Tournament,Post-Season Tournament Sorting Index
0,2015,Kentucky,Southeastern Conference (SEC),Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,...,13.00,10.99,John Calipari,14 years,15.0,36.9,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
1,2021,Gonzaga,West Coast Conference (WCC),West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,...,24.63,19.49,Mark Few,24 years,25.0,36.5,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
2,2024,UConn,Big East Conference,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,...,20.55,15.33,Dan Hurley,5 years,6.0,36.4,1,East,March Madness,1
3,2008,Kansas,Big 12 Conference,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,...,13.90,12.87,Bill Self,20 years,21.0,35.2,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
4,2019,Virginia,Atlantic Coast Conference (ACC),Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,...,17.84,13.59,Tony Bennett,14 years,15.0,34.2,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5


In [8]:
# Con estas dos limpiezas, ya hemos pasado de un dataset de 144 columnas, a uno de 75
# Vamos a ver los tipos de nuestras columnas
print(NCAA_df.dtypes)
tipo_resumen = NCAA_df.dtypes.value_counts()
print("Tipos de Datos más comunes: ", tipo_resumen)

Season                                    int64
Team Name                                object
Conference                               object
Historical Conference                    object
Adjusted Temo                           float64
                                         ...   
Net Rating                              float64
Seed                                     object
Region                                   object
Post-Season Tournament                   object
Post-Season Tournament Sorting Index      int64
Length: 75, dtype: object
Tipos de Datos más comunes:  float64    65
object      8
int64       2
Name: count, dtype: int64


In [9]:
# Vamos a intentar reducir más la cantidad de inputs
NCAA_df = NCAA_df.iloc[:, :-4]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7697, 71)


,Season,Team Name,Conference,Historical Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [10]:
# Seguimos eliminando columnas, ahora vamos con la 3 columna, que es similar a la segunda, Historical Conference
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Historical')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7697, 70)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [11]:
# Seguimos eliminando columnas, ahora vamos con unas columnas que contienen NST que no nos aportan demasiada información
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('NST|DFP')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7697, 67)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [12]:
# Seguimos eliminando columnas, ahora vamos con puntos ofensivos y defensivos, ya que nos interesa ir a los porcentajes sobretodo
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('PtFG')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7697, 63)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [13]:
# Seguimos eliminando columnas, ahora vamos con las columnas que contienen Rate
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Rate')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7697, 56)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [14]:
# Seguimos eliminando columnas, ahora vamos con unas columnas que contienen información sobre los entrenadores
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Coach|coach')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7697, 53)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,34.47,20.03,5.69,6.08,27.92,30.08,18.01,13.00,10.99,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,19.89,6.54,22.61,11.41,23.99,18.56,13.28,24.63,19.49,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,18.66,20.29,13.83,14.05,25.88,17.21,20.98,20.55,15.33,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,31.72,22.11,5.59,5.77,26.38,28.48,18.37,13.90,12.87,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2


In [15]:
# Seguimos eliminando columnas, ahora vamos con las columnas que contienen Pts
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Pts|pts')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7697, 48)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,34.47,20.03,5.69,6.08,27.92,30.08,18.01,13.00,10.99,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,19.89,6.54,22.61,11.41,23.99,18.56,13.28,24.63,19.49,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,18.66,20.29,13.83,14.05,25.88,17.21,20.98,20.55,15.33,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,31.72,22.11,5.59,5.77,26.38,28.48,18.37,13.90,12.87,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2


In [16]:
# Vamos a dejar las columnas que sean porcentajes, divididos entre 100
pct_columns = [col for col in NCAA_df.columns if "Pct" in col]

# Paso 2: Dividir los valores de estas columnas por 100
NCAA_df[pct_columns] = NCAA_df[pct_columns] / 100

# El dataframe ahora tiene las columnas "Pct" divididas por 100
NCAA_df.head()

<ipython-input-16-86ad85c96ddc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NCAA_df[pct_columns] = NCAA_df[pct_columns] / 100


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,0.515116,0.162529,...,34.47,20.03,5.69,6.08,27.92,30.08,18.01,13.00,10.99,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,0.610178,0.161279,...,19.89,6.54,22.61,11.41,23.99,18.56,13.28,24.63,19.49,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,0.569075,0.144997,...,18.66,20.29,13.83,14.05,25.88,17.21,20.98,20.55,15.33,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,0.565912,0.191079,...,31.72,22.11,5.59,5.77,26.38,28.48,18.37,13.90,12.87,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,0.551800,0.147067,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2


In [17]:
# Vamos a ver los tipos de nuestras columnas finales
print(NCAA_df.dtypes)
tipo_resumen_v2 = NCAA_df.dtypes.value_counts()
print("Tipos de Datos más comunes: ", tipo_resumen_v2)

Season                               int64
Team Name                           object
Conference                          object
Adjusted Temo                      float64
Adjusted Offensive Efficiency      float64
Adjusted Defensive Efficiency      float64
Avg Possession Length (Offense)    float64
Avg Possession Length (Defense)    float64
eFGPct                             float64
TOPct                              float64
ORPct                              float64
OffFT                              float64
DefFT                              float64
Tempo                              float64
AdjTempo                           float64
OE                                 float64
AdjOE                              float64
DE                                 float64
AdjDE                              float64
AdjEM                              float64
FG2Pct                             float64
FG3Pct                             float64
FTPct                              float64
BlockPct   

In [18]:
# Vamos a ver que variedad de años tenemos, para quedarnos con los últimos años
NCAA_df['Season'].value_counts()

,count
Season,
2024,362
2023,360
2022,353
2021,351
2020,348
2019,346
2017,341
2018,341
2016,340


In [19]:
#Como vemos todos los años están bastante parecidos, y tenemos datos desde el 2002. Vamos a eliminar los datos del 2002, 2003 y 2004 para reducir algo mas el dataset y usar los últimos 20 años
NCAA_df = NCAA_df[~NCAA_df['Season'].isin([2002, 2003, 2004])]
NCAA_df['Season'].value_counts()

,count
Season,
2024,362
2023,360
2022,353
2021,351
2020,348
2019,346
2018,341
2017,341
2016,340


In [20]:
# Vamos a ver el número de líneas que nos quedan
print("Tamaño del dataset:", NCAA_df.shape)

Tamaño del dataset: (6767, 48)


In [21]:
# Vamos a eliminar todas las lineas que tengan algún tipo de caracter en blanco
NCAA_df = NCAA_df.replace(pd.NA, regex=True).dropna()
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()


Tamaño del dataset: (6767, 48)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,0.515116,0.162529,...,34.47,20.03,5.69,6.08,27.92,30.08,18.01,13.00,10.99,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,0.610178,0.161279,...,19.89,6.54,22.61,11.41,23.99,18.56,13.28,24.63,19.49,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,0.569075,0.144997,...,18.66,20.29,13.83,14.05,25.88,17.21,20.98,20.55,15.33,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,18.6,17.5,0.565912,0.191079,...,31.72,22.11,5.59,5.77,26.38,28.48,18.37,13.90,12.87,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,0.551800,0.147067,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2


In [22]:
# Vamos a cruzar el dataset de estadísticas con el dataset de finalistas del MarchMadness


# URL de la página web que contiene la tabla
url = 'https://www.ncaa.com/history/basketball-men/d1'

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    # Analizar el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar la tabla en el HTML (ajusta el selector según sea necesario)
    table = soup.find('table')

    # Leer la tabla HTML en un DataFrame de pandas
    df = pd.read_html(str(table))[0]

    # Exportar el DataFrame a un archivo CSV
    df.to_csv('table_data.csv', index=False)

# Abro la tabla
Campeones = pd.read_csv('table_data.csv')

#Me quedo solamente con las 5 primeras columnas
Campeones = Campeones.iloc[:, :5]

# Voy a poner el mismo nombre de columnas
Campeones.columns = ['Season', 'Team Name', 'Coach', 'Score', 'Subcampeon']
print(Campeones)

    Season                 Team Name             Coach  Score  \
0     2024              UConn (37-3)        Dan Hurley  75-60   
1     2023              UConn (31-8)        Dan Hurley  76-59   
2     2022             Kansas (34-6)         Bill Self  72-69   
3     2021             Baylor (28-2)        Scott Drew  86-70   
4     2020  Canceled due to Covid-19                --     --   
..     ...                       ...               ...    ...   
81    1943            Wyoming (31-2)   Everett Shelton  46-34   
82    1942           Stanford (28-4)      Everett Dean  53-38   
83    1941          Wisconsin (20-3)        Bud Foster  39-34   
84    1940            Indiana (20-3)  Branch McCracken  60-42   
85    1939             Oregon (29-5)     Howard Hobson  46-33   

          Subcampeon  
0             Purdue  
1    San Diego State  
2     North Carolina  
3            Gonzaga  
4                 --  
..               ...  
81        Georgetown  
82         Dartmouth  
83  Washingt

<ipython-input-22-5bc2ce9f9cbd>:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


In [23]:
# Voy a quitar el record del equipo
import re

# Función para limpiar los nombres de los equipos
def clean_team_name(name):
    # Elimina el contenido entre paréntesis y los paréntesis mismos
    return re.sub(r'\s*\(.*?\)', '', name).strip()

# Aplicar la función a la columna 'Team Name'
Campeones['Team Name'] = Campeones['Team Name'].apply(clean_team_name)
print(Campeones)

    Season                 Team Name             Coach  Score  \
0     2024                     UConn        Dan Hurley  75-60   
1     2023                     UConn        Dan Hurley  76-59   
2     2022                    Kansas         Bill Self  72-69   
3     2021                    Baylor        Scott Drew  86-70   
4     2020  Canceled due to Covid-19                --     --   
..     ...                       ...               ...    ...   
81    1943                   Wyoming   Everett Shelton  46-34   
82    1942                  Stanford      Everett Dean  53-38   
83    1941                 Wisconsin        Bud Foster  39-34   
84    1940                   Indiana  Branch McCracken  60-42   
85    1939                    Oregon     Howard Hobson  46-33   

          Subcampeon  
0             Purdue  
1    San Diego State  
2     North Carolina  
3            Gonzaga  
4                 --  
..               ...  
81        Georgetown  
82         Dartmouth  
83  Washingt

In [24]:
Finalistas_df = pd.concat([
    Campeones[['Season', 'Team Name']].rename(columns={'Team Name': 'Team Name', 'Season': 'Season'}).assign(Is_Champion='Si'),
    Campeones[['Season', 'Subcampeon']].rename(columns={'Subcampeon': 'Team Name'}).assign(Is_Champion='No')
])
print(Finalistas_df)

    Season                 Team Name Is_Champion
0     2024                     UConn          Si
1     2023                     UConn          Si
2     2022                    Kansas          Si
3     2021                    Baylor          Si
4     2020  Canceled due to Covid-19          Si
..     ...                       ...         ...
81    1943                Georgetown          No
82    1942                 Dartmouth          No
83    1941          Washington State          No
84    1940                    Kansas          No
85    1939                Ohio State          No

[172 rows x 3 columns]


In [25]:
# Cambiar Connecticut por Central Connecticut
Finalistas_df['Team Name'] = Finalistas_df['Team Name'].replace('Connecticut', 'Central Connecticut')
print(Finalistas_df)

    Season                 Team Name Is_Champion
0     2024                     UConn          Si
1     2023                     UConn          Si
2     2022                    Kansas          Si
3     2021                    Baylor          Si
4     2020  Canceled due to Covid-19          Si
..     ...                       ...         ...
81    1943                Georgetown          No
82    1942                 Dartmouth          No
83    1941          Washington State          No
84    1940                    Kansas          No
85    1939                Ohio State          No

[172 rows x 3 columns]


In [26]:
# Vamos a unir las dos tablas
MarchMad_Campeones = pd.merge(NCAA_df, Finalistas_df, on=['Season', 'Team Name'], how='left')
MarchMad_Campeones.fillna("No Llega", inplace=True)
print(MarchMad_Campeones)

      Season                 Team Name  \
0       2015                  Kentucky   
1       2021                   Gonzaga   
2       2024                     UConn   
3       2008                    Kansas   
4       2019                  Virginia   
...      ...                       ...   
6762    2009                      NJIT   
6763    2015                 Grambling   
6764    2005          Maryland-Eastern   
6765    2021  Mississippi Valley State   
6766    2013                 Grambling   

                                  Conference  Adjusted Temo  \
0              Southeastern Conference (SEC)           62.4   
1                West Coast Conference (WCC)           73.8   
2                        Big East Conference           64.6   
3                          Big 12 Conference           66.8   
4            Atlantic Coast Conference (ACC)           59.4   
...                                      ...            ...   
6762       American East Conference (AmEast)          

In [27]:
# Vamos a ver los tipos de nuestras columnas finales
print(MarchMad_Campeones.dtypes)
tipo_resumen_v3 = MarchMad_Campeones.dtypes.value_counts()
print("Tipos de Datos más comunes: ", tipo_resumen_v3)
print("Tamaño del dataset:", MarchMad_Campeones.shape)

Season                               int64
Team Name                           object
Conference                          object
Adjusted Temo                      float64
Adjusted Offensive Efficiency      float64
Adjusted Defensive Efficiency      float64
Avg Possession Length (Offense)    float64
Avg Possession Length (Defense)    float64
eFGPct                             float64
TOPct                              float64
ORPct                              float64
OffFT                              float64
DefFT                              float64
Tempo                              float64
AdjTempo                           float64
OE                                 float64
AdjOE                              float64
DE                                 float64
AdjDE                              float64
AdjEM                              float64
FG2Pct                             float64
FG3Pct                             float64
FTPct                              float64
BlockPct   

In [28]:
# Por último, vamos a renombrar el df, para tenerlo listo para los siguientes pasos
March_Madness_df = MarchMad_Campeones

# Define la ruta a la carpeta
folder_path = "/content/drive/MyDrive/Colab Notebooks/TFM"

# Crea la carpeta si no existe
os.makedirs(folder_path, exist_ok=True)

March_Madness_df.to_csv("/content/drive/MyDrive/Colab Notebooks/TFM/March_Madness_df.csv")



Tras ejecutar la última línea de código guardo una copia en GitHub para almacenar el Notebook.